# ブラウザオブザーバビリティとセッションリプレイ 

## 概要

このチュートリアルでは、AgentCore Browser セッションにオブザーバビリティを追加し、ブラウザコンソールログ、ネットワークログ、送信された CDP コマンド、およびブラウザ上で実行されたエージェントアクションを表示する方法を学びます。 


### チュートリアル詳細


| 項目         | 詳細                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ       | 会話型                                                                   |
| エージェントタイプ          | シングル                                                                           |
| エージェントフレームワーク   | Nova Act                                                                         |
| LLM モデル           | Amazon Nova Act モデル                                                            |
| チュートリアルコンポーネント | AgentCore Browser コンソールでのブラウザログの観察                                |
| チュートリアル分野   | 垂直統合                                                                         |
| 難易度  | 簡単                                                                             |
| 使用 SDK            | Amazon Bedrock AgentCore Python SDK、boto3 SDK、Nova Act                          |

### チュートリアルアーキテクチャ

このチュートリアルでは、ブラウザコンソールログ、ネットワークログ、CDP コマンド、およびブラウザ上で実行されたエージェントアクションを観察する方法を説明します。   


### チュートリアルの主な機能

* Browser ツールのセッション記録を有効化 
* Nova Act と Browser ツールの連携
* ログとセッションリプレイの観察

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Amazon boto3 SDK
* Nova Act SDK および API キー - https://nova.amazon.com/act にアクセスして API キーを生成してください

In [ ]:
!pip install  -r requirements.txt --quiet

## 記録を有効にしたカスタム AgentCore Browser リソースの作成
デフォルトの Browser ツールは記録が有効になっていないため、まず記録を有効にした Browser ツールリソースを作成する必要があります。その後、この Browser リソースを使用してブラウザセッションを開始します。 

In [ ]:
## Create an S3 bucket to store browser recordings
## If you want to use an existing bucket, skip this step and update the bucket name in the next step.
import boto3
import uuid
from boto3.session import Session

boto_session = Session()

region = boto_session.region_name
s3_client = boto3.client('s3', region_name=region)

bucket_name = f"agentcore-browser-recordings-{str(uuid.uuid4())[:8]}"
s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
print(f"S3 バケットを作成しました: {bucket_name}")

### Browser ツールリソース作成に必要な権限を持つ実行ロールの作成

適切な権限を持つ実行ロールを作成するためのヘルパーユーティリティを作成しましょう。 

In [ ]:
## Create execution role with permisions to create browser 
def create_agentcore_role(agent_name):
    iam_client = boto3.client('iam')
    agentcore_role_name = f'agentcore-{agent_name}-role'
    boto_session = Session()
    region = boto_session.region_name
    account_id = boto3.client("sts").get_caller_identity()["Account"]
    
    role_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "BrowserPermissions",
                "Effect": "Allow",
                "Action": [
                    "bedrock-agentcore:ConnectBrowserAutomationStream",
                    "bedrock-agentcore:ListBrowsers",
                    "bedrock-agentcore:GetBrowserSession",
                    "bedrock-agentcore:ListBrowserSessions",
                    "bedrock-agentcore:CreateBrowser",
                    "bedrock-agentcore:StartBrowserSession",
                    "bedrock-agentcore:StopBrowserSession",
                    "bedrock-agentcore:ConnectBrowserLiveViewStream",
                    "bedrock-agentcore:UpdateBrowserStream",
                    "bedrock-agentcore:DeleteBrowser",
                    "bedrock-agentcore:GetBrowser"
                ],
                "Resource": "*"
            },
            {
                "Sid": "S3Permissions",
                "Effect": "Allow",
                "Action": [
                    "s3:PutObject",
                    "s3:GetObject",
                    "s3:ListBucket"
                ],
                "Resource": [
                    f"arn:aws:s3:::{bucket_name}",
                    f"arn:aws:s3:::{bucket_name}/*"
                ]
            },
            {
                "Sid": "CloudWatchLogsPermissions",
                "Effect": "Allow",
                "Action": [
                    "logs:CreateLogGroup",
                    "logs:CreateLogStream",
                    "logs:PutLogEvents",
                    "logs:DescribeLogStreams"
                ],
                "Resource": "*"
            }
        ]
    }
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "AssumeRolePolicy",
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock-agentcore.amazonaws.com"
                },
                "Action": "sts:AssumeRole",
                "Condition": {
                    "StringEquals": {
                        "aws:SourceAccount": f"{account_id}"
                    },
                    "ArnLike": {
                        "aws:SourceArn": f"arn:aws:bedrock-agentcore:{region}:{account_id}:*"
                    }
                }
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(
        assume_role_policy_document
    )
    role_policy_document = json.dumps(role_policy)

    try:
        # Create IAM Role with the trust policy
        agentcore_iam_role = iam_client.create_role(
            RoleName=agentcore_role_name,
            AssumeRolePolicyDocument=assume_role_policy_document_json,
        )
        print(f"ロール {agentcore_role_name} が正常に作成されました。")

        # Attach the inline permissions policy to the role
        iam_client.put_role_policy(
            RoleName=agentcore_role_name,
            PolicyName=f'{agentcore_role_name}-inline-policy',
            PolicyDocument=role_policy_document
        )
        print(f"インラインポリシーをロール {agentcore_role_name} にアタッチしました。")

    except iam_client.exceptions.EntityAlreadyExistsException:
        print("ロールが既に存在します -- 削除して再作成します")
        
        # Detach and delete any existing inline policies
        policies = iam_client.list_role_policies(RoleName=agentcore_role_name)
        for policy_name in policies['PolicyNames']:
            iam_client.delete_role_policy(
                RoleName=agentcore_role_name,
                PolicyName=policy_name
            )
        
        # Delete and re-create the role
        print(f"ロール {agentcore_role_name} を削除中...")
        iam_client.delete_role(RoleName=agentcore_role_name)
        print(f"ロール {agentcore_role_name} を再作成中...")
        
        agentcore_iam_role = iam_client.create_role(
            RoleName=agentcore_role_name,
            AssumeRolePolicyDocument=assume_role_policy_document_json
        )
        print(f"ロール {agentcore_role_name} が正常に再作成されました。")

        # Re-attach the inline permissions policy to the re-created role
        iam_client.put_role_policy(
            RoleName=agentcore_role_name,
            PolicyName=f'{agentcore_role_name}-inline-policy',
            PolicyDocument=role_policy_document
        )
        print(f"インラインポリシーをロール {agentcore_role_name} に再アタッチしました。")

    # Pause to make sure changes propagate
    time.sleep(10)
    
    return agentcore_iam_role

### 記録を有効にした Browser ツールリソースの作成

In [ ]:
## Use boto3 to create a custom browser tool resource with recording enabled
import boto3
import time
import json
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

cp_client = boto3.client('bedrock-agentcore-control', region_name=region)

# Define parameters for the Browser Tool
browser_name = "my_custom_browser"
browser_description = "Test browser for observability and session replay"
execution_role = create_agentcore_role(browser_name)
execution_role_arn = execution_role['Role']['Arn'] # Todo: Replace with your IAM role ARN
s3_bucket_name = bucket_name # Replace with your S3 bucket name if you have an existing bucket
s3_prefix = "replay-data"

try:
    response = cp_client.create_browser(
        name=browser_name,
        description=browser_description,
        networkConfiguration={
            "networkMode": "PUBLIC" # Or "VPC" if you need VPC integration
        },
        executionRoleArn=execution_role_arn,
        clientToken=str(uuid.uuid4()), # Unique token for idempotency
        recording={
            "enabled": True,
            "s3Location": {
                "bucket": s3_bucket_name,
                "prefix": s3_prefix
            }
        }
    )
    print(response)
    print(f"ブラウザツールを正常に作成しました: {response['browserId']}")
    browserId = response['browserId']
except cp_client.exceptions.ConflictException as e:
    print("この名前のブラウザツールは既に存在します。別の名前を選択してください。")

## Nova Act スクリプトの作成
Nova Act は、前のステップで作成した Browser ツールリソースを使用してブラウザセッションを開始し、その上でブラウザアクションを実行します。

In [ ]:
%%writefile basic_browser_with_nova_act.py
"""Amazon Bedrock AgentCore と Nova Act を使用したブラウザ自動化スクリプト。

このスクリプトは、AI を活用した Web 自動化を実演します：
- Amazon Bedrock AgentCore を通じてブラウザセッションを初期化
- 自然言語による Web 操作のために Nova Act に接続
- ブラウザを使用した自動検索とデータ抽出を実行
"""

from bedrock_agentcore.tools.browser_client import browser_session , BrowserClient
from nova_act import NovaAct
from rich.console import Console
import argparse
import json

console = Console()

from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print("使用リージョン:", region)

def browser_with_nova_act(prompt, starting_page, nova_act_key,  browserId, region="us-west-2"):
    result = None
    
    browser_client = BrowserClient(region)
    browser_client.start(identifier=browserId) # ここで作成した Browser ツール ID を使用
    
    ws_url, headers = browser_client.generate_ws_headers()
    try:
        with NovaAct(
            cdp_endpoint_url=ws_url,
            cdp_headers=headers,
            nova_act_api_key=nova_act_key,
            starting_page=starting_page,
        ) as nova_act:
            result = nova_act.act(prompt)
    except Exception as e:
        console.print(f"NovaAct エラー: {e}")

    finally:
        browser_client.stop()
        return result


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--prompt", required=True, help="ブラウザ検索の指示")
    parser.add_argument("--starting-page", required=True, help="開始 URL")
    parser.add_argument("--nova-act-key", required=True, help="Nova Act API キー")
    parser.add_argument("--region", default="us-west-2", help="AWS リージョン")
    parser.add_argument("--browserID", required=True, help="使用する Browser ツール ID")
    args = parser.parse_args()

    result = browser_with_nova_act(
        args.prompt, args.starting_page, args.nova_act_key, args.browserID, args.region
    )
    console.print(f"\n[cyan] レスポンス[/cyan] {result.response}")
    console.print(f"\n[bold green]Nova Act 結果:[/bold green] {result}")

#### スクリプトの実行
スクリプトを実行する前に、以下に Nova Act API キーを貼り付けてください。 

In [ ]:
NOVA_ACT_KEY= '' ### Paste your Nova Act Key here

In [ ]:
!python basic_browser_with_nova_act.py --prompt "Search for macbooks and extract the details of the first one" --starting-page "https://www.amazon.com/" --browserID {browserId} --nova-act-key {NOVA_ACT_KEY}

## AgentCore Browser コンソールでのオブザーバビリティ
* スクリプトの実行中に、AWS コンソール：https://us-west-2.console.aws.amazon.com/bedrock-agentcore/builtInTools にアクセスできます。異なるリージョンで実行している場合は、この URL のリージョンを置き換えてください。
* 「Browser use tools」タブをクリックします。
* 「my-custom-browser」をクリックします。セッションがまだ実行中の場合はライブビューへのリンク、または記録を表示するリンクが表示されます。
* セッションが実行中の場合は終了するまで待ち、その後「記録を表示」をクリックします。
以下のようなページが表示されます

![image](./images/browser_recording_1.png)

* #### 記録されたブラウザセッションをリプレイできます 
* #### セッション中に訪問した各ページを検査できます 
* #### アクションタブでエージェントが実行したアクションを表示できます 
* #### ページ DOM の詳細、コンソールログ、ブラウザに送信された CDP コマンド、ネットワークログを表示できます 
* #### さらなるデバッグのために各ログをダウンロードできます
* #### アクションタブの各アクションの「表示」をクリックして、ブラウザで実行された正確なアクションを確認できます 
* #### 例：任意の「クリック」アクションタイプを表示し、ブラウザ上の赤い円に注目してください。これはクリックアクションが発生した正確な位置を示しています。 

# おめでとうございます、探索をお楽しみください！ 